In [38]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np
from sklearn.neural_network import MLPClassifier

In [27]:
## Change the name to get your path
user = 'Yash'
file1 = 'AG01-01.csv'
file2 = 'GTH01-01.csv'

In [28]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [112]:
def useable_Data(data, gt):
    data['start.time'] = pd.to_datetime(data['start.time'])
    observedData = data[(data['start.time'] >= '2017-10-03 16:44:00') & (data['start.time'] <= '2017-10-03 18:44:00')]
    observedData['index'] = range(len(observedData))
    gt['index'] = gt['time']
    observedData = pd.merge(observedData,gt[['index','coding']])
    observedData.sort_index(inplace=True)
    copy = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']].copy()
    copy.loc[-1] = copy.loc[0]  # adding a row
    copy.index = copy.index + 1  # shifting index
    copy.sort_index(inplace=True)
    copy.columns = 'last.' + copy.columns
    observedData = pd.concat([observedData, copy], axis = 1)
    observedData = observedData.drop(observedData.index[len(observedData)-1])
    return observedData
#     return copy

In [113]:
data = pd.read_csv(load_Data(user, file1), header=0)
gt = pd.read_csv(load_Data(user, file2), header=0)
observedData = pd.DataFrame(useable_Data(data, gt))
observedData.head()

/Users/yashmehra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,index,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,start.time,coding,last.mean.vm,last.sd.vm,last.mean.ang,last.sd.ang,last.p625,last.dfreq,last.ratio.df
0,0.0,1.009207,0.008657,-49.775885,0.842458,0.147968,9.230769,0.091802,2017-10-03 16:44:00,non-sedentary,1.009207,0.008657,-49.775885,0.842458,0.147968,9.230769,0.091802
1,1.0,1.007030,0.012245,-47.654547,1.862073,0.074857,3.076923,0.106718,2017-10-03 16:44:00,non-sedentary,1.009207,0.008657,-49.775885,0.842458,0.147968,9.230769,0.091802
2,2.0,1.008842,0.009694,-46.577448,0.651607,0.146617,8.205128,0.109818,2017-10-03 16:44:00,non-sedentary,1.007030,0.012245,-47.654547,1.862073,0.074857,3.076923,0.106718
3,3.0,1.007484,0.013547,-47.424062,0.790202,0.129838,2.051282,0.107984,2017-10-03 16:44:00,non-sedentary,1.008842,0.009694,-46.577448,0.651607,0.146617,8.205128,0.109818
4,4.0,1.009535,0.006776,-47.288191,0.558104,0.064207,8.205128,0.119769,2017-10-03 16:44:00,non-sedentary,1.007484,0.013547,-47.424062,0.790202,0.129838,2.051282,0.107984


In [114]:
observedData = observedData.sample(frac = 1)

train = observedData[:5000]
test = observedData[5000:]

## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
X = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
y = observedData['coding']

trainX = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
trainY = train['coding']
testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
testY = test['coding']

## SVM

In [115]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [117]:
clf.score(testX,testY)

0.7288810260946483

## Neural Network

In [ ]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(X,y)